<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [ ]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [44]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Molokai10.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Molokai10.zip
   creating: /content/CCAP/content/Molokai10/
  inflating: /content/CCAP/content/Molokai10/2010_Molokai_CCAP.tif  
  inflating: /content/CCAP/content/Molokai10/2010_Molokai_LS.tif  
   creating: /content/CCAP/content/Molokai10/Images/
  inflating: /content/CCAP/content/Molokai10/Images/1_1.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_1025.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_1537.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_2049.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_2561.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_3073.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_3585.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_4097.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_4609.tif  
  inflating: /content/CCAP/content/Molokai10/Images/1_5121.tif  
  inflating: /conten

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [45]:
# original images and labels folder: how many files are there?

# Count how many files are in the images folder
org_img = '/content/CCAP/content/Molokai10/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')

# Count how many files are in the labels folder
org_label = '/content/CCAP/content/Molokai10/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1080
The number of files in the folder is: 1080


In [ ]:
# # original images and labels folder: are any of them duplicated?

# # Check the images folder for duplicates
# org_img = '/content/CCAP/content/Kauai10/Images'
# file_hashes = {}
# for org_img_filename in os.listdir(org_img):
#     file_path = os.path.join(org_img, org_img_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {org_img_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = org_img_filename
# print("Duplicate check completed.")

# # Check the label folder for duplicates
# org_label = '/content/CCAP/content/Kauai10/Labels'
# file_hashes = {}
# for org_label_filename in os.listdir(org_label):
#     file_path = os.path.join(org_label, org_label_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {org_label_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = org_label_filename
# print("Duplicate check completed.")

Duplicate check completed.
Duplicate check completed.


In [46]:
df = pd.read_csv('/content/CCAP/content/Molokai10/Molokai10.csv')

In [47]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,673774.792178,675003.549951,2.349134e+06,2.350363e+06,0,1.0
1,1025,513,513,1,2,675003.581956,676232.397422,2.349134e+06,2.350363e+06,0,1.0
2,1537,513,1025,1,3,676232.402366,677461.226108,2.349134e+06,2.350363e+06,0,1.0
3,2049,513,1537,1,4,677461.191368,678689.981511,2.349134e+06,2.350363e+06,0,1.0
4,2561,513,2049,1,5,678690.051385,679918.820215,2.349134e+06,2.350363e+06,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
3857,25601,9851,25089,9339,1076,733985.987134,735214.794851,2.326723e+06,2.327952e+06,0,1.0
3858,26113,9851,25601,9339,1077,735214.804523,736443.648874,2.326723e+06,2.327952e+06,0,1.0
3859,26625,9851,26113,9339,1078,736443.564339,737672.387927,2.326723e+06,2.327952e+06,0,1.0
3860,27137,9851,26625,9339,1079,737672.401853,738901.225655,2.326723e+06,2.327952e+06,0,1.0


In [48]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [49]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
291,15361,2049,14849,1537,192,709410.008128,710638.851059,2.345448e+06,2.346676e+06,15,0.005547
937,21505,3585,20993,3073,366,724155.624966,725384.439310,2.341761e+06,2.342990e+06,13,0.003975
938,21505,3585,20993,3073,366,724155.624966,725384.439310,2.341761e+06,2.342990e+06,14,0.000534
946,22017,3585,21505,3073,367,725384.429760,726613.215039,2.341761e+06,2.342990e+06,13,0.014740
947,22017,3585,21505,3073,367,725384.429760,726613.215039,2.341761e+06,2.342990e+06,14,0.003555
...,...,...,...,...,...,...,...,...,...,...,...
3726,20481,9217,19969,8705,958,721698.017204,722926.824289,2.328244e+06,2.329473e+06,17,0.005634
3727,20481,9217,19969,8705,958,721698.017204,722926.824289,2.328244e+06,2.329473e+06,18,0.014267
3733,20993,9217,20481,8705,959,722926.798467,724155.564062,2.328244e+06,2.329473e+06,16,0.004665
3734,20993,9217,20481,8705,959,722926.798467,724155.564062,2.328244e+06,2.329473e+06,17,0.012745


In [50]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [51]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
291,15361,2049,14849,1537,192,709410.008128,710638.851059,2.345448e+06,2.346676e+06,15,0.005547,14849_1537.tif
937,21505,3585,20993,3073,366,724155.624966,725384.439310,2.341761e+06,2.342990e+06,13,0.003975,20993_3073.tif
938,21505,3585,20993,3073,366,724155.624966,725384.439310,2.341761e+06,2.342990e+06,14,0.000534,20993_3073.tif
946,22017,3585,21505,3073,367,725384.429760,726613.215039,2.341761e+06,2.342990e+06,13,0.014740,21505_3073.tif
947,22017,3585,21505,3073,367,725384.429760,726613.215039,2.341761e+06,2.342990e+06,14,0.003555,21505_3073.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
3726,20481,9217,19969,8705,958,721698.017204,722926.824289,2.328244e+06,2.329473e+06,17,0.005634,19969_8705.tif
3727,20481,9217,19969,8705,958,721698.017204,722926.824289,2.328244e+06,2.329473e+06,18,0.014267,19969_8705.tif
3733,20993,9217,20481,8705,959,722926.798467,724155.564062,2.328244e+06,2.329473e+06,16,0.004665,20481_8705.tif
3734,20993,9217,20481,8705,959,722926.798467,724155.564062,2.328244e+06,2.329473e+06,17,0.012745,20481_8705.tif


In [52]:
!mkdir '/content/CCAP/Molokai10'
!mkdir '/content/CCAP/Molokai10/Images'
!mkdir '/content/CCAP/Molokai10/Labels'

In [ ]:
# Copy the useable images files
# Define the source and destination directories
source_images = '/content/CCAP/content/Molokai10/Images'
destination_images = '/content/CCAP/Molokai10/Images'

# Iterate through the 'tif_name' column in the subset_df
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)

    try:
        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files
# Define the source and destination directories
source_labels = '/content/CCAP/content/Molokai10/Labels'
destination_labels = '/content/CCAP/Molokai10/Labels'

# Iterate through the 'tif_name' column in the subset_df
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)

    try:
        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [55]:
# NEW images and labels folder: how many files are there?

# Count how many files are in the new images folder
new_img = '/content/CCAP/Molokai10/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')

# Count how many files are in the new labels folder
new_label = '/content/CCAP/Molokai10/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 135
The number of files in the folder is: 135


In [ ]:
# # new images and labels folder: are any of them duplicated?

# # Check the new images folder for duplicates
# new_img = '/content/CCAP/content/Molokai/Images'
# file_hashes = {}
# for new_img_filename in os.listdir(new_img):
#     file_path = os.path.join(new_img, new_img_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {new_img_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = new_img_filename
# print("Duplicate check completed.")

# # Check the new label folder for duplicates
# new_label = '/content/CCAP/content/Molokai/Labels'
# file_hashes = {}
# for new_label_filename in os.listdir(new_label):
#     file_path = os.path.join(new_label, new_label_filename)
#     if os.path.isfile(file_path):
#         with open(file_path, 'rb') as file:
#             file_content = file.read()
#             file_hash = hashlib.md5(file_content).hexdigest()
#         if file_hash in file_hashes:
#             print(f"Duplicate found: {new_label_filename} is a duplicate of {file_hashes[file_hash]}")
#         else:
#             file_hashes[file_hash] = new_label_filename
# print("Duplicate check completed.")

Duplicate check completed.
Duplicate check completed.


In [56]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Molokai10_subset.csv'

# Export the DataFrame to a CSV file
subset_df.to_csv(csv_file_path, index=False)

print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Molokai10_subset.csv


In [57]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/CCAP/Molokai10'

# Define the path for the ZIP file
zip_file_path = '/content/Molokai10_subset.zip'

# Create a ZIP archive of the folder
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)

print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Molokai10_subset.zip


In [58]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'

# Move the ZIP file to Google Drive
shutil.move(zip_file_path, drive_folder)

print(f'ZIP file moved to Google Drive: {drive_folder}')

# Move the CSV file to Google Drive
shutil.move(csv_file_path, drive_folder)

print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
